In [6]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt
import random 
import tensorflow as tf
import numpy as np
import math

/Users/mohammfa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
#find all values greater than a certain value
def findGreater(mat,val):
    arr = np.array([])
    for i in range(mat.shape[0]): #xaxis
        for j in range(mat.shape[1]): #yaxix
            if(mat[i,j] > val):
                arr = np.append(arr,mat[i,j])
                
    return arr

In [8]:
"""Def Generate neutrino 
Random origin (x,y)
Random direction (dx, dy) or (one angles)
Random energy (0 to 1) or (10 to 300 nanoseconds)
Random time
(make code adjustable)
We want to know how fast the neutrino is"""
def generateNeutrino():
    x = random.uniform(0,10)
    y = random.uniform(0,10)
    angle = random.vonmisesvariate(0, 0)
    energy = random.random();
    time = random.randint(10,300)

    return x, y, angle, energy, time

In [9]:
def generateSignal(x,y,angle,energy,time):
    nodes = 10
    x_arr = np.linspace(0,nodes,nodes + 1)
    y_arr = np.linspace(0,nodes,nodes + 1)
    #10 by 10 grid which is the VICE CUBE
    
    #using MeshGrid
    mesh_x, mesh_y = np.meshgrid(x_arr, y_arr)

    #find distance of each sensor to the origion using euclidean distance formula
    x_sq = mesh_x - x
    y_sq = mesh_y - y
    x_sq = np.power(x_sq,2)
    y_sq = np.power(y_sq,2)
    distance = np.sqrt(x_sq + y_sq)
    
    #find charge
    #charge formula: C = (maxDistance - distance_to_each_node)^2 / maxDistance
    maxDistance = math.sqrt((nodes * nodes) + (nodes * nodes))
    charge = np.power((maxDistance - distance),2) / pow(maxDistance,2)
    
    
    #based on the angle and the origin create a line (data fit) that would simulate a neutrino path
    return mesh_x, mesh_y, distance, charge

In [10]:
def exampleRun():
    x,y,angle,energy,time = generateNeutrino();
    mesh_x, mesh_y, distance, charge = generateSignal(x,y,angle,energy,time)
    #print("x\n", mesh_x)
    #print("y\n", mesh_y)

    plt.plot(mesh_x,mesh_y, marker='.', color='k', linestyle='none')
    
    
    CS = plt.contourf(mesh_x, mesh_y,charge, 15, cmap=plt.cm.rainbow,
                  vmax=charge.max(), vmin=0)
    plt.colorbar()  
    plt.show()

    print("x y angle energy time")
    print(x,y,angle,energy,time)
    
    #print("distance\n", distance)
    print("charge\n", charge)

In [11]:
#decompose signal function
"""" function to decompose the signal to create a training vector and a training label
def decomposeSignal():
    """
#plan is to use have a vector of which sensors detected the neutrino and then have a vector for distance, charge
#then use that as the training data

def createTrainingVector(dataSize):
    #we changed from array of arrays to 2D array
    #we changed data types
    #also we changed feature columns shape to specify how many features we have
    xy = np.zeros(2)
    trainVec = np.zeros((dataSize,121),dtype = np.float32)
    trainLabel = np.zeros(dataSize, dtype = np.float32)

    for i in range(dataSize):
        x_o,y_o,angle,energy,time = generateNeutrino();
        mesh_x, mesh_y, distance, charge = generateSignal(x_o,y_o,angle,energy,time)
        #xy[0] = x_o
        #xy[1] = y_o
        #flattening the charge matrix to a vector
        chargeVector = charge.flatten()
        trainVec[i,:] = chargeVector
        trainLabel[i] = x_o
    
    return trainVec, trainLabel

In [12]:
def inputFunction(datasize=10):
    trainVec, trainLabel = createTrainingVector(dataSize)    
    features = {'x' : trainVec}
    #trainLabel is the output. which currently is the origin
    labels = trainLabel
    #labels = np.int(labels)
    
    
    return features, labels

In [13]:
#for some reason this does not work. I however found another approach with using tf.estimator.inputs.numpy_input_fn
def trainInputFn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    print(type(features))


    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))


    # Shuffle, repeat, and batch the examples.
    return dataset.shuffle(1000).repeat().batch(batch_size)

In [14]:
#if __name__== "__main__":
dataSize = 120
features, labels = inputFunction(dataSize)

print((features['x'].dtype))
print(labels.dtype)
print(features['x'].shape)
print(labels.shape)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x=(features),
                                                    y=(labels),num_epochs=None,shuffle=True)

my_feature_columns = []
for key in features.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print("feature_columns=",my_feature_columns)

# Specify that all features have real-value data
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[121])]

# Build 3 layer DNN with 10, 20, 10 units respectively.
regressor = tf.estimator.DNNRegressor(feature_columns=my_feature_columns,
                                      hidden_units=[10, 20, 10])

regressor.train(input_fn=train_input_fn, steps=1000)

"""
print(train_input_fn)
"""

float32
float32
(120, 121)
(120,)
feature_columns= [_NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/gn/y5s233mx34b21b029nk5sc3h0000gn/T/tmphku8i792', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1056ec860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized

'\nprint(train_input_fn)\n'

In [5]:
!ls


README.md               iCEcUBE-Copy2.ipynb     iris_training.csv
helloWorldExample.ipynb iCEcUBE.ipynb
iCEcUBE-Copy1.ipynb     iris_test.csv
